# Preprocessing the taxi data - Intentionally Blank

**NOTE: This notebook does not need to be executed. A copy of the preprocessed dataframe is saved as a [parquet file](https://parquet.apache.org/)**. For the preparation notebook click [here](./prep.ipynb)

Before preparing and cleaning the taxi dataset, we should first preprocess the CSV to make it smaller. We collected the data from the Chicago Data Portal. We filtered the original dataset by the trip_start_timestamp directly via the API to minimize the initial filesize. To get all the trips from 2016 we used the following query: https://data.cityofchicago.org/resource/wrvz-psew.csv?$where=trip_start_timestamp%20between%20%272016-01-01T00:00:00%27%20and%20%20%272016-12-31T23:59:59%27&$limit=1000000000.
<br>For further information about the dataset and the API click the following link: [Chicago Data Portal - Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew).

To run this notebook you need to download the dataset from the aforementioned link. And copy it to the "data" folder as "original_taxi_data.csv". Furthermore atleast 16GB of ram is needed to read the CSV and 32GB is recommended because the initial CSV is ~14GB big.

**Dependencies:**
- Pandas
- Pyarrow (conda install pyarrow)
  - Needed for saving to a parquet file

In [1]:
# Importing the libraries
import pandas as pd
import os  
os.makedirs('./data', exist_ok=True) 

In [2]:
# Reading the csv file
# Note: This file is not included in the repository due to its size. Please use the link above to download the file. Loading the file may take a few minutes.
taxi_df = pd.read_csv('data/original_taxi_data.csv')
taxi_df['trip_start_timestamp'] = pd.to_datetime(taxi_df['trip_start_timestamp'])
taxi_df['trip_end_timestamp'] = pd.to_datetime(taxi_df['trip_end_timestamp'])

In [3]:
# Checking for the right time range
taxi_df["trip_start_timestamp"].min(), taxi_df["trip_start_timestamp"].max()

(Timestamp('2016-01-01 00:00:00'), Timestamp('2016-12-31 23:45:00'))

In [4]:
# Checking memory usage for later comparison
taxi_df.memory_usage(deep=True)

Index                                128
trip_id                       3080655883
taxi_id                       5875028507
trip_start_timestamp           254074712
trip_end_timestamp             254074712
trip_seconds                   254074712
trip_miles                     254074712
pickup_census_tract            254074712
dropoff_census_tract           254074712
pickup_community_area          254074712
dropoff_community_area         254074712
fare                           254074712
tips                           254074712
tolls                          254074712
extras                         254074712
trip_total                     254074712
payment_type                  2034421040
company                       2256911991
pickup_centroid_latitude       254074712
pickup_centroid_longitude      254074712
pickup_centroid_location      2692429277
dropoff_centroid_latitude      254074712
dropoff_centroid_longitude     254074712
dropoff_centroid_location     2681476484
dtype: int64

In [5]:
taxi_df.head(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,223789629c9e0a01fbab0d787d2664ccdb8355c0,507b1e4d1f39a8a26e7249e6a627f5a0c798dfdafa7b16...,2016-12-31 23:45:00,2016-12-31 23:45:00,180.0,0.7,NaN,NaN,NaN,NaN,...,1.0,5.75,Cash,City Service,NaN,NaN,NaN,NaN,NaN,NaN
1,a1d390b16ede0f133408103b79dcb56bbd74365e,73b2f5adecea91eeef3900303a07f1b0519a594cffb6b0...,2016-12-31 23:45:00,2017-01-01 00:15:00,2160.0,5.4,NaN,NaN,NaN,NaN,...,0.0,23.50,Cash,Chicago Taxicab,NaN,NaN,NaN,NaN,NaN,NaN
2,2fffdf0e5b45125ed3fd7027b92e31bd7e7085ef,d41ab2be597b82c3e6b0b0ecccf98883a84db0d9aed4f6...,2016-12-31 23:45:00,2017-01-01 00:00:00,1080.0,5.1,NaN,NaN,NaN,NaN,...,0.0,15.75,Cash,City Service,NaN,NaN,NaN,NaN,NaN,NaN
3,3c1d5e90e522f7be0bf92c96f5164360d8d02f94,24515782c70f09819506a7724a57e77c78fea60c4dc91d...,2016-12-31 23:45:00,2017-01-01 00:00:00,780.0,2.9,NaN,NaN,NaN,NaN,...,0.0,11.00,Cash,Sun Taxi,NaN,NaN,NaN,NaN,NaN,NaN
4,d9046368ad0f1ba4cc27c659e9467cd3602bd458,f1eda6f0cb8e48e7fdb5f623a4a5113a84c159fbf73638...,2016-12-31 23:45:00,2016-12-31 23:45:00,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,5.00,Credit Card,Suburban Dispatch LLC,NaN,NaN,NaN,NaN,NaN,NaN


We delete irrelevant columns to save as much memory as possible.

In [6]:
#Drop columns pickup_centroid_location, dropoff_centroid_location, fare, tips, tolls, extras, payment_type, pickup_community_area, dropoff_community_area, company
taxi_df = taxi_df.drop(columns=['pickup_centroid_latitude', 'pickup_centroid_longitude', 'dropoff_centroid_latitude', 'dropoff_centroid_longitude', 'fare', 'tips', 'tolls', 'extras', 'payment_type', 'pickup_community_area', 'dropoff_community_area', 'company'])

We preemptively delete rows where the trip_end_timestamp, trip_start_timestamp and taxi_ids columns have null values, because to compute the idle seconds, there should not be any null values in the time stamp.

In [7]:
taxi_df[taxi_df["trip_end_timestamp"].isnull() | taxi_df["trip_start_timestamp"].isnull() | taxi_df["taxi_id"].isnull()]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location
404,6c616d9bd36d367fee0ec951f1510d9afdea8249,NaN,2016-12-31 23:45:00,2017-01-01 00:00:00,600.0,0.0,1.703108e+10,1.703108e+10,6.75,POINT (-87.6188683546 41.8909220259),POINT (-87.6288741572 41.8920726347)
2873,96842033a8a7dd35681c452479dbe9795f5d7e6f,e7e187c80ff0f05f971fef2ac660198b4e86ccecae67e7...,2016-12-31 22:45:00,NaT,NaN,0.0,NaN,NaN,0.00,POINT (-87.771166703 41.9788295262),NaN
2902,0ee2a90ea42e24b272180bccd26c29f1300acff6,NaN,2016-12-31 22:45:00,2016-12-31 23:00:00,1200.0,0.0,1.703108e+10,1.703108e+10,15.25,POINT (-87.6188683546 41.8909220259),POINT (-87.6318639497 41.8920421365)
5694,d6354aa062c3245e39ad7cb8c0063fafca52bfcb,NaN,2016-12-31 21:45:00,2016-12-31 22:15:00,1320.0,0.0,1.703108e+10,1.703108e+10,12.75,POINT (-87.6188683546 41.8909220259),POINT (-87.6129454143 41.8919715078)
6440,fff8f42d31d886b242bb03077ed285f5deb71843,NaN,2016-12-31 21:30:00,2016-12-31 21:45:00,240.0,0.0,1.703108e+10,1.703108e+10,4.50,POINT (-87.6188683546 41.8909220259),POINT (-87.6262149064 41.8925077809)
...,...,...,...,...,...,...,...,...,...,...,...
31709369,b438457d94b43f7c9eb85a61c8618eede325fb12,89ee7f39a29ed33f083ce17d20e0d9f7a07528a5188084...,2016-01-01 11:30:00,NaT,NaN,0.0,1.703183e+10,NaN,0.00,POINT (-87.717503858 41.942859303),NaN
31716791,d40e9fe9b72dfa210478bda8150137e9dfcd7fb1,94024afd53bfce6f81da57630f32bebb2242ef299c70ea...,2016-01-01 07:15:00,NaT,NaN,0.0,NaN,NaN,0.00,POINT (-87.7215590627 41.968069),NaN
31723301,25caee01f98a77edc18f5332f86969573703f04d,NaN,2016-01-01 04:15:00,2016-01-01 04:15:00,0.0,0.0,NaN,NaN,92.00,NaN,NaN
31723302,f99474b30b65e483ca5ceb8c892269c2a12b08c0,NaN,2016-01-01 04:15:00,2016-01-01 04:15:00,0.0,0.0,NaN,NaN,45.00,NaN,NaN


Now we check if the trip_seconds, trip_miles or trip_total column has rows with values below or equal to zero.

In [8]:
print(len(taxi_df[(taxi_df['trip_seconds'] <= 0) | (taxi_df['trip_miles'] <= 0) | (taxi_df['trip_total'] <= 0)]))
taxi_df[(taxi_df['trip_seconds'] <= 0) | (taxi_df['trip_miles'] <= 0) | (taxi_df['trip_total'] <= 0)].head(5)

5509823


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location
4,d9046368ad0f1ba4cc27c659e9467cd3602bd458,f1eda6f0cb8e48e7fdb5f623a4a5113a84c159fbf73638...,2016-12-31 23:45:00,2016-12-31 23:45:00,0.0,0.0,NaN,NaN,5.00,NaN,NaN
6,7ae0d20727ec680d5b25e467c58e65442df10574,3b5ccf9479decf48844e1beab2b5dba14e75c17972c396...,2016-12-31 23:45:00,2017-01-01 00:00:00,780.0,0.0,NaN,NaN,12.00,NaN,NaN
8,8ef8dd979bf31342527f50437861dcb882c41f7c,3fb54b9070b703bd28358e704311d7fe8afb6950881654...,2016-12-31 23:45:00,2017-01-01 00:15:00,1680.0,0.0,NaN,NaN,20.25,NaN,NaN
10,e0463a944364b59e6eaf5eaced95be69bb8e1879,3d288150e092eea073ce7ba66c4c9ad75f8c572a62f080...,2016-12-31 23:45:00,2016-12-31 23:45:00,0.0,0.0,NaN,NaN,7.50,NaN,NaN
15,69e9408d443c59e4a0afeee31d165382a67d1d9d,9d916e6060ccc4440f82b50495548dab39cb6ea8ea4e71...,2016-12-31 23:45:00,2017-01-01 00:00:00,480.0,0.0,1.703108e+10,1.703108e+10,9.00,POINT (-87.6188683546 41.8909220259),POINT (-87.6188683546 41.8909220259)


Because trips which have no length in time and distance and have no revenue generated we see them as outliers without any informational value, we drop them from the dataframe.

In [9]:
taxi_df = taxi_df[(taxi_df['trip_seconds'] > 0) & (taxi_df['trip_miles'] > 0) & (taxi_df['trip_total'] > 0)]
taxi_df.reset_index(drop=True, inplace=True)
taxi_df

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location
0,223789629c9e0a01fbab0d787d2664ccdb8355c0,507b1e4d1f39a8a26e7249e6a627f5a0c798dfdafa7b16...,2016-12-31 23:45:00,2016-12-31 23:45:00,180.0,0.70,NaN,NaN,5.75,NaN,NaN
1,a1d390b16ede0f133408103b79dcb56bbd74365e,73b2f5adecea91eeef3900303a07f1b0519a594cffb6b0...,2016-12-31 23:45:00,2017-01-01 00:15:00,2160.0,5.40,NaN,NaN,23.50,NaN,NaN
2,2fffdf0e5b45125ed3fd7027b92e31bd7e7085ef,d41ab2be597b82c3e6b0b0ecccf98883a84db0d9aed4f6...,2016-12-31 23:45:00,2017-01-01 00:00:00,1080.0,5.10,NaN,NaN,15.75,NaN,NaN
3,3c1d5e90e522f7be0bf92c96f5164360d8d02f94,24515782c70f09819506a7724a57e77c78fea60c4dc91d...,2016-12-31 23:45:00,2017-01-01 00:00:00,780.0,2.90,NaN,NaN,11.00,NaN,NaN
4,9d75355d1917ea38154a1076ff2d6dcb92cec86c,a8107c5481ea5eed4d6fa93cd1648f3f69c6822a6fff49...,2016-12-31 23:45:00,2017-01-01 00:30:00,2952.0,10.41,NaN,NaN,37.75,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
26248440,b3e944c1ca206a01322abc79ffa1d59dc44f2dfe,46c8c622009ad9edf747b4a249509ecb361cfea350d6df...,2016-01-01 00:00:00,2016-01-01 00:30:00,1200.0,0.10,1.703184e+10,1.703132e+10,13.05,POINT (-87.6241352979 41.84924675450001),POINT (-87.6209929134 41.8849871918)
26248441,c548f9274722a68c7e1e7164a354e253129cffc3,ffa006c14e54b463189c69f05ad3af2b89d20bb60e68b3...,2016-01-01 00:00:00,2016-01-01 00:15:00,420.0,0.10,NaN,NaN,8.05,POINT (-87.592310855 41.794090253),POINT (-87.6178596758 41.8129489392)
26248442,24872670ed67d12a5f227d063e5fcc2217a9f66c,e7edb97b460627c86c83be13c595325e390ea4153cdabd...,2016-01-01 00:00:00,2016-01-01 00:15:00,420.0,1.00,1.703108e+10,1.703184e+10,7.05,POINT (-87.6318639497 41.8920421365),POINT (-87.6327464887 41.8809944707)
26248443,a84bf68a891b35dd5e7ff7e5a6214cab03a4116a,b0f4966a7a652995c265e8bd0611316732b732c0aeffe6...,2016-01-01 00:00:00,2016-01-01 00:15:00,420.0,0.70,1.703108e+10,1.703108e+10,7.65,POINT (-87.6188683546 41.8909220259),POINT (-87.6378442095 41.8932163595)


In [10]:
taxi_df.dropna(subset=['trip_end_timestamp', 'trip_start_timestamp', 'taxi_id'], axis=0, inplace=True)

In [11]:
# Sort the taxi data by the start timestamp
taxi_df = taxi_df.sort_values(['trip_start_timestamp'])

# Reset the index
taxi_df = taxi_df.reset_index(drop=True)

taxi_df.set_index(["taxi_id", taxi_df.index], inplace=True)

In [12]:
idle_seconds = pd.Series()
i = 0
for id in taxi_df.index.get_level_values(0).unique():
    idle_seconds = pd.concat([idle_seconds, taxi_df.loc[id, "trip_start_timestamp"] - taxi_df.loc[id, "trip_end_timestamp"].shift(1)])
idle_seconds.name = "idle_seconds"
idle_seconds = idle_seconds.dt.total_seconds()

In [13]:
taxi_df.set_index([taxi_df.index.get_level_values(1)], inplace=True)

In [14]:
idle_seconds

0                 NaN
3909           1800.0
2080343     2812500.0
2083338        2700.0
2092718        7200.0
              ...    
26224328        900.0
26225927       2700.0
26230356       4500.0
26231582        900.0
26232808       1800.0
Name: idle_seconds, Length: 26247185, dtype: float64

In [15]:
taxi_df = taxi_df.merge(idle_seconds, left_index=True, right_index=True, how='left')

Deleting rows with null values and duplicates is done in this notebook instead of the preparation notebook to ensure that most computers with low memory can run the preparation notebook.

In [16]:
display(taxi_df[taxi_df.isnull().any(axis = 1)])

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location,idle_seconds
0,869d4dbf2d7df18738ed7ba777d50d3699099c52,2016-01-01 00:00:00,2016-01-01 00:15:00,900.0,2.2,1.703128e+10,1.703108e+10,11.65,POINT (-87.642648998 41.8792550844),POINT (-87.6129454143 41.8919715078),NaN
1,f729decdd42d2a418b747b07a43a0916d3117a5d,2016-01-01 00:00:00,2016-01-01 00:15:00,360.0,0.6,1.703108e+10,1.703132e+10,10.05,POINT (-87.6262149064 41.8925077809),POINT (-87.6209929134 41.8849871918),NaN
2,8d93c76a6a31e137c080678dae07593ab99bd77d,2016-01-01 00:00:00,2016-01-01 00:15:00,1080.0,6.5,1.703108e+10,1.703105e+10,18.65,POINT (-87.6291051864 41.9002212967),POINT (-87.6761824959 41.95054569650001),NaN
3,007ed9c92ce247e557009c5ce9c09a0ba8f798a9,2016-01-01 00:00:00,2016-01-01 00:15:00,1260.0,3.9,1.703184e+10,1.703108e+10,17.20,POINT (-87.6241352979 41.84924675450001),POINT (-87.6321092196 41.9002656868),NaN
4,288deb4c98c9681f7d5d02e478a94ae46571fddb,2016-01-01 00:00:00,2016-01-01 00:00:00,240.0,0.4,1.703108e+10,1.703108e+10,6.45,POINT (-87.6378442095 41.8932163595),POINT (-87.6318639497 41.8920421365),NaN
...,...,...,...,...,...,...,...,...,...,...,...
26247176,caefa0bac780a47e89f3bea5d567e720a46afec3,2016-12-31 23:45:00,2017-01-01 00:00:00,840.0,3.0,NaN,NaN,15.00,POINT (-87.6333080367 41.899602111),POINT (-87.6559981815 41.9442266014),900.0
26247177,732c2595112b29f0316be606d7f33c8e86dd53f8,2016-12-31 23:45:00,2016-12-31 23:45:00,300.0,1.3,NaN,NaN,7.50,POINT (-87.6838349425 41.9477915865),POINT (-87.6559981815 41.9442266014),-900.0
26247179,0ddaf6bf0128e1181b27f64f7bc7c367ea793097,2016-12-31 23:45:00,2016-12-31 23:45:00,480.0,1.6,NaN,NaN,10.25,POINT (-87.69915534320002 41.9227606205),POINT (-87.6763559892 41.90120699410001),0.0
26247181,255c838d7d1d988968d9493776b60dff54874d9d,2016-12-31 23:45:00,2017-01-01 00:00:00,180.0,0.5,NaN,NaN,6.25,POINT (-87.7112105933 41.93866619620001),POINT (-87.69915534320002 41.9227606205),900.0


In [17]:
# Drop rows with missing values
taxi_df = taxi_df.dropna(how='any', axis=0, subset=taxi_df.columns.difference(['idle_seconds']))
print(f"Number of rows after deleting rows with null values: {len(taxi_df)} ")

Number of rows after deleting rows with null values: 17364075 


In [18]:
display(taxi_df[taxi_df.duplicated(subset=['trip_start_timestamp', 'trip_end_timestamp', 'trip_id'])].head(5))

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location,idle_seconds


Because no duplicate rows exist in our dataset we skip the deletion part and drop the obsolete trip_id column.

In [19]:
taxi_df = taxi_df.drop(columns=[ "trip_id"])

In [20]:
# Convert trip_seconds to uint32 without losing information
taxi_df = taxi_df.astype({'trip_seconds': 'uint32', 'pickup_census_tract': 'int64', 'dropoff_census_tract': 'int64'})

In [21]:
# Last look at the data
taxi_df.head(5)

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location,idle_seconds
0,2016-01-01,2016-01-01 00:15:00,900,2.2,17031281900,17031081402,11.65,POINT (-87.642648998 41.8792550844),POINT (-87.6129454143 41.8919715078),NaN
1,2016-01-01,2016-01-01 00:15:00,360,0.6,17031081500,17031320100,10.05,POINT (-87.6262149064 41.8925077809),POINT (-87.6209929134 41.8849871918),NaN
2,2016-01-01,2016-01-01 00:15:00,1080,6.5,17031081100,17031050600,18.65,POINT (-87.6291051864 41.9002212967),POINT (-87.6761824959 41.95054569650001),NaN
3,2016-01-01,2016-01-01 00:15:00,1260,3.9,17031841000,17031081000,17.20,POINT (-87.6241352979 41.84924675450001),POINT (-87.6321092196 41.9002656868),NaN
4,2016-01-01,2016-01-01 00:00:00,240,0.4,17031081800,17031081700,6.45,POINT (-87.6378442095 41.8932163595),POINT (-87.6318639497 41.8920421365),NaN


In [22]:
# Optional: If you want to save the preprocessed data as a csv file uncomment the following line
# taxi_df.to_csv('data/taxi_data_preprocessed.csv', index=False)

# Saving the preprocessed data as a parquet file with gzip compression
taxi_df.to_parquet('data/taxi_data_preprocessed.gzip', compression='gzip')